# Building the Darknet/YOLO Framework
- Using the [C Code Run Darknet/YOLO repo](https://codeberg.org/CCodeRun/darknet#table-of-contents)
- Based on https://codeberg.org/CCodeRun/darknet#linux-cmake-method
- See the [Darknet/YOLO FAQ](https://www.ccoderun.ca/programming/yolo_faq/)
- Last updated by Stéphane Charette on 2025-08-15 for Darknet v4.0-52
- Remember to select:  Edit -> Notebook settings -> T4 GPU

# Install the dependencies

In [ ]:
# First part will install the tools we'll use to build.
!sudo apt-get install build-essential git libopencv-dev cmake

# Which NVIDIA libraries are currently installed?
!dpkg -l | egrep "libcuda|libcudnn"

# Last updated on 2025-08-15; you may need to update the library version numbers
# You should have libcudnn and libcuda installed.  If not, then you'll need to
# check what is available, and install it with the following lines.

#!apt-cache search libcudnn
#!apt-cache search cuda-libraries
#!sudo apt-get install libcudnn9-dev-XYZ cuda-libraries-dev-XYZ

# Verifying NVIDIA

In [ ]:
# There are 2 specific tools from NVIDIA we're going to use to verify that everything is installed.

# The first is nvidia-smi which gives real-time information in the GPU.
!nvidia-smi

# Next is the nvidia compiler.  If you get a "command not found" error then things will not work!
!nvcc --version

# Cloning the Darknet/YOLO repo

In [ ]:
# Remove any pre-existing repo files, and clone the Darknet/YOLO repo.
%mkdir -p ~/src
%cd ~/src
%rm -rf ~/src/darknet
!git clone https://codeberg.org/CCodeRun/darknet.git

# Preparing to build the Darknet/YOLO repo

In [ ]:
# Prepare the Darknet/YOLO build directory
%cd ~/src/darknet
%rm -rf ~/src/darknet/build
%mkdir ~/src/darknet/build
%cd ~/src/darknet/build
!cmake -DCMAKE_BUILD_TYPE=Release ..

# Once cmake has finished, make sure that CUDA and cuDNN are working.  The output should contain lines such as:
#     "CUDA detected. Darknet will use the GPU."
# and
#     "Found cuDNN"

# Building and installing Darknet/YOLO files

In [ ]:
# Build Darknet/YOLO
%cd ~/src/darknet/build
!make -j $(nproc)

In [ ]:
# Confirm that the Darknet CLI was built correctly
%cd ~/src/darknet/build
!src-cli/darknet version

In [ ]:
# Create and install the .deb installation package
%cd ~/src/darknet/build
!make package
!sudo dpkg -i darknet*.deb

In [ ]:
# Confirm that we can run "darknet" from anywhere since it has now been installed
%cd ~
!darknet version
# Show some of the /opt/ files we've installed.  This should show ~40 .cfg files.
!ls -lh /opt/darknet/cfg/yolov*

# Building and installing DarkHelp

In [ ]:
# Build and install the DarkHelp CLI

!sudo apt-get install build-essential libtclap-dev libmagic-dev libopencv-dev
%cd ~/src
%rm -rf ~/src/DarkHelp
!git clone https://codeberg.org/CCodeRun/DarkHelp.git
%cd ~/src/DarkHelp
%mkdir build
%cd build
!cmake -DCMAKE_BUILD_TYPE=Release ..
!make -j $(nproc)
!make package
!sudo dpkg -i darkhelp*.deb

# Test the MSCOCO Weights

In [ ]:
# Download the MSCOCO pre-trained weights (https://codeberg.org/CCodeRun/darknet#mscoco-pre-trained-weights)
%cd ~/src/darknet
!wget --no-clobber https://github.com/hank-ai/darknet/releases/download/v2.0/yolov4-tiny.weights

# This is what the command looks like when using Darknet to predict.  Because we're using --dont-show, the results will be saved as an image to "predictions.jpg"
#!darknet detector test --dont-show cfg/coco.data cfg/yolov4-tiny.cfg yolov4-tiny.weights artwork/dog.jpg

# And this how we do the equivalent but with DarkHelp instead.  See the DarkHelp documentation to see all the options which alter the look of the output image.
# https://www.ccoderun.ca/darkhelp/api/Parameters.html
!DarkHelp --json --autohide off --keep --duration off --fontscale 0.7 --line 5 --shade 0.25 --threshold 0.5 --outdir . cfg/yolov4-tiny.cfg cfg/coco.names yolov4-tiny.weights artwork/dog.jpg

# Display the results
import cv2 as cv2
from matplotlib import pyplot as plt

img = cv2.imread('dog.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


# All done!

At this point, you'd need to upload your dataset, and then call `darknet detector train ...` to begin training your neural network.

Also see:
- https://www.ccoderun.ca/programming/yolo_faq/#training_command
- https://codeberg.org/CCodeRun/darknet#training
